In [ ]:
'''
PIL和skimage对图像处理的区别：
PIL提供了最基本的数字图像处理，skimage和numpy一样，以数组的形式进行数据处理
PIL的数据格式（width,height,channel）,skimage是（height,width,channel）
‘’‘

In [1]:
from skimage import color
import openslide
from openslide import deepzoom
import numpy as np
import math
import PIL
from PIL import Image
import os
from skimage import filters,transform,io,color
import cv2

In [2]:
# 将svs格式图片缩小32倍，存储为png格式，用PIL库
def slide_to_png(slide):
    scale_factor = 32
    base_path = os.path.abspath(slide)
    print(base_path)
    slide = openslide.OpenSlide(slide)
    orig_w,orig_h = slide.dimensions
    new_w = math.floor(orig_w / scale_factor)
    new_h = math.floor(orig_h / scale_factor)
    level = slide.get_best_level_for_downsample(scale_factor)
    orig_slide = (slide.read_region((0,0),level,slide.level_dimensions[level])).convert("RGB")
    img = orig_slide.resize((new_w,new_h),PIL.Image.BILINEAR)
    save_path = os.path.join('.'+'/file/',os.path.split(base_path)[1].split('.')[0]+'.png')
    print(save_path)
    img.save(save_path)
    return img,new_w,new_h

In [ ]:
img,_,_ = slide_to_png('1.svs')
img.show()

/home/jn/code/lianxi/1.svs


In [ ]:
# 将svs存为jpeg格式，缩小32倍，用skimage；
def slide_to_jpeg(slide):
    scale_factor=32
    base_path = os.path.abspath(slide)
    slide = openslide.OpenSlide(slide)
    orig_w,orig_h = slide.dimensions
    new_w = math.floor(orig_w / scale_factor)
    new_h = math.floor(orig_h / scale_factor)
    level = slide.get_best_level_for_downsample(scale_factor)
    orig_slide = (slide.read_region((0,0),level,slide.level_dimensions[level])).convert("RGB")
    im = np.array(orig_slide)
    im = transform.resize(im,(new_h,new_w))
    # 这里im数组中数据范围是[0,1]，*255后转换成RGB值
    im = (im*255).astype("uint8")
    print(im.dtype)
    save_path = os.path.join('.'+'/file/',os.path.split(base_path)[1].split('.')[0]+'1.jpeg')
    io.imsave(save_path,im)
    return im,new_h,new_w

# 注意：这里的im是numpy数组格式的

In [ ]:
img1,_,_ = slide_to_jpeg('test.tif')
io.imshow(img1)

In [ ]:
Slide = openslide.OpenSlide("./file/3.svs")
#Slide1 = deepzoom.DeepZoomGenerator(Slide,tile_size = 256,overlap = 1,limit_bounds = False)
#print(Slide1.level_count)

In [ ]:
Slide.level_count

In [ ]:
np_slide = np.array((Slide.read_region((0,0),10,Slide.level_dimensions[2])).convert("RGB"))

In [ ]:
hsv_slide = color.rgb2hsv(np_slide)
print(hsv_slide.shape)
print(hsv_slide[0].shape)
print(hsv_slide[1].shape)
print(hsv_slide[2].shape)

In [ ]:
img_R = np_slide[:,:,0]
img_G = np_slide[:,:,1]
img_B = np_slide[:,:,2]
R_thresh_value = filters.threshold_otsu(img_R)
G_thresh_value = filters.threshold_otsu(img_G)
B_thresh_value = filters.threshold_otsu(img_B)
otsu_R = (img_R > R_thresh_value)
otsu_G = (img_G > G_thresh_value)
otsu_B = (img_B > B_thresh_value)
tissue_RGB = np.logical_not(otsu_R & otsu_G & otsu_B)
tissue_h = (hsv_slide[:,:,1] > filters.threshold_otsu(hsv_slide[:,:,1]))
tissue_mask = tissue_RGB & tissue_h
hsv_slide = np_slide * np.dstack([tissue_mask,tissue_mask,tissue_mask])

In [ ]:
tissue_mask = np.save('./3.npy',tissue_mask)

In [ ]:
(rows,cols) = Slide.dimensions
print(rows,cols)

In [ ]:
width = 256
height = 256
tiles_cols = math.ceil(cols / width)
tiles_rows = math.ceil(rows / height)
print(tiles_rows,tiles_cols)

In [ ]:
indices = list()
for r in range(0,tiles_rows):
    start_r = r * width
    end_r = (r + 1) * width  if(r < tiles_rows-1) else rows
    for c in range(0,tiles_cols):
        start_c = c * height
        end_c = (c + 1) * height if(c < tiles_cols-1) else cols
        indices.append((start_r,end_r,start_c,end_c,r+1,c+1))

In [ ]:
for i in indices:
    np_img = hsv_slide[i[0]:i[1],i[2]:i[3],:]
    np_sum = np_img[:,:,0] + np_img[:,:,1] + np_img[:,:,2]
    tissue_percentage = np.count_nonzero(np_sum) / (np_sum.size+0.1)
    if tissue_percentage >= 0.50:
        print(tissue_percentage)
        im = Image.fromarray(np_img)
        im.save(os.path.join("./file/masked",str(i)+".png"))